Calculo de Coorelação:

Imports que importam :) 

In [24]:
import numpy as np
import math

Carregar valores(teste):

In [25]:
gerax = lambda base,n,dif: [base + x*dif for x in range(n)] # modo preguiçoso de gerar n números espaçados em 0.5

x = gerax(2.5, 6,0.5)
y = [0.4,0.4,0.3,0.2,0.2,0.1]

Somatórios:

In [26]:
# n
n = len(x)
#somas
xysum = np.dot(x,y) #soma dos x*y

xsum = sum(x)
ysum = sum(y)
#soma dos quadrados
x2sum = np.dot(x,x) #multiplica entrada a entrada dos vetores x e ele mesmo e soma
y2sum = sum([v*v for v in y]) #forma na unha de fazer o mesmo
#quadrado das somas:
xsum2 = xsum**2
ysum2 = ysum**2

In [27]:
#parte superior:
sup = (xysum - xsum*ysum/n)
print(sup)

-0.5499999999999998


In [30]:
#parte inferior:
infe = math.sqrt ( (x2sum - xsum2/n) * (y2sum - ysum2/n) )
print(infe)

0.5664215155988811


In [31]:
corr = sup/infe
print("Taxa de correlação entre X e Y é de {}".format(corr))

Taxa de correlação entre X e Y é de -0.9710083124552239
